#**Predicting the Severity Of Accidents **

In [ ]:
# importing data on colab.
from google.colab import files
import io
uploaded = files.upload()

Saving testset_final.xlsx to testset_final.xlsx
Saving trainset_final.xlsx to trainset_final.xlsx


In [ ]:
import pandas as pd
import io

trainset_final = pd.read_excel(io.BytesIO(uploaded['trainset_final.xlsx']))
testset_final = pd.read_excel(io.BytesIO(uploaded['testset_final.xlsx']))
# checking the type of features
print(trainset_final.dtypes)
print(testset_final.dtypes)

accident_index               object
accident_severity             int64
vehicle_reference             int64
vehicle_type                  int64
towing_and_articulation       int64
vehicle_left_hand_drive       int64
journey_purpose_of_driver     int64
sex_of_driver                 int64
age_of_driver                 int64
engine_capacity_cc            int64
propulsion_code               int64
age_of_vehicle                int64
driver_home_area_type         int64
driver_imd_decile             int64
dtype: object
accident_index               object
accident_severity             int64
vehicle_reference             int64
vehicle_type                  int64
towing_and_articulation       int64
vehicle_left_hand_drive       int64
journey_purpose_of_driver     int64
sex_of_driver                 int64
age_of_driver                 int64
engine_capacity_cc            int64
propulsion_code               int64
age_of_vehicle                int64
driver_home_area_type         int64
driver_imd_dec

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

cat_columns = ['vehicle_type', 'towing_and_articulation', 'journey_purpose_of_driver', 'sex_of_driver', 'propulsion_code', 'driver_home_area_type', 'driver_imd_decile']

# Using the OneHotEncoder function
encoder = OneHotEncoder(sparse=False, drop='first')

# Fiting and transforming the training data
encoded_train = encoder.fit_transform(trainset_final[cat_columns])
# Transforming the test data
encoded_test = encoder.transform(testset_final[cat_columns])

# Creating DataFrame for encoded features (for training data)
encoded_train_df = pd.DataFrame(encoded_train, columns=encoder.get_feature_names_out(cat_columns))
# And for test data
encoded_test_df = pd.DataFrame(encoded_test, columns=encoder.get_feature_names_out(cat_columns))

# Reseting index to ensure concatenation works correctly
trainset_final.reset_index(drop=True, inplace=True)
testset_final.reset_index(drop=True, inplace=True)

# Concatenating the encoded columns back to the original dataframe
trainset_final_encoded = pd.concat([trainset_final.drop(cat_columns, axis=1), encoded_train_df], axis=1)
testset_final_encoded = pd.concat([testset_final.drop(cat_columns, axis=1), encoded_test_df], axis=1)

encoder = OneHotEncoder(sparse_output=False, drop='first')

print(trainset_final_encoded.head())


  accident_index  accident_severity  vehicle_reference  \
0  2022010415662                  2                  1   
1  2022221179602                  3                  2   
2  2022010357122                  3                  1   
3  2022010368509                  3                  1   
4  2022010361741                  3                  1   

   vehicle_left_hand_drive  age_of_driver  engine_capacity_cc  age_of_vehicle  \
0                        1             45                 124               2   
1                        1             66                1995               5   
2                        1             49                 124              10   
3                        1             34                 125               0   
4                        1             40                 898               3   

   vehicle_type_3  vehicle_type_4  vehicle_type_5  ...  \
0             1.0             0.0             0.0  ...   
1             0.0             0.0             0.

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


#**Seprating predictors and target**

In [ ]:
# Separating for the training data
y_train = trainset_final_encoded["accident_severity"].copy()
X_train = trainset_final_encoded.drop("accident_severity", axis=1)

# Separating for the test data
y_test = testset_final_encoded["accident_severity"].copy()
X_test = testset_final_encoded.drop("accident_severity", axis=1)


#**Baseline**



In [ ]:
# Droping the 'accident_index' column from both the training and testing datasets
X_train = trainset_final.drop(['accident_index', 'accident_severity'], axis=1)
y_train = trainset_final['accident_severity']

X_test = testset_final.drop(['accident_index', 'accident_severity'], axis=1)
y_test = testset_final['accident_severity']


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support

# Runing the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Fiting the model on the training data
rf_model.fit(X_train, y_train)

# Predicting on the test data
y_pred = rf_model.predict(X_test)

# Calculating precision, recall, and F1-score
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

# Print the results
print("Random Forest:")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F score: {fscore:.3f}")


Random Forest:
Precision: 0.354
Recall: 0.339
F score: 0.317


Precision 0.354: 35.4% of all instances predicted to be at a specific accident severity level were accurate.
Recall 0.339: 33.9% of actual instances of a given accident severity level are identified correctly.
The F1-score, which stands for 0.317.The harmonic mean of the precision and recall in this instance is 0.317, which shows that the two variables are balanced but still leaves potential for improvement.

#**Model Developement**

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from datetime import timedelta
from time import time

# Starting the timer
start = time()

# Applying the Decision Tree classifier
dt = DecisionTreeClassifier(random_state=7)

# Defining hyperparameter grid
hp_grid = {
    'max_depth': [5, 10, 15, 20, 25, 30, 35, 40],
    'min_samples_split': [5, 10, 15, 20, 25, 30, 35],
}

# Defining scorer
scorer = make_scorer(f1_score, average='macro')

# Applying GridSearchCV
grid_search = GridSearchCV(dt, hp_grid, cv=5, scoring=scorer, return_train_score=True, verbose=2)

# Fitting GridSearchCV
grid_search.fit(X_train, y_train)

# Print the best parameters and execution time
print("Best Parameters:", grid_search.best_params_)
print("Execution time HH:MM:SS:", timedelta(seconds=time() - start))


Fitting 5 folds for each of 56 candidates, totalling 280 fits
[CV] END ...................max_depth=5, min_samples_split=5; total time=   0.0s
[CV] END ...................max_depth=5, min_samples_split=5; total time=   0.0s
[CV] END ...................max_depth=5, min_samples_split=5; total time=   0.0s
[CV] END ...................max_depth=5, min_samples_split=5; total time=   0.0s
[CV] END ...................max_depth=5, min_samples_split=5; total time=   0.0s
[CV] END ..................max_depth=5, min_samples_split=10; total time=   0.0s
[CV] END ..................max_depth=5, min_samples_split=10; total time=   0.0s
[CV] END ..................max_depth=5, min_samples_split=10; total time=   0.0s
[CV] END ..................max_depth=5, min_samples_split=10; total time=   0.0s
[CV] END ..................max_depth=5, min_samples_split=10; total time=   0.0s
[CV] END ..................max_depth=5, min_samples_split=15; total time=   0.0s
[CV] END ..................max_depth=5, min_sam

In [ ]:
grid_search.best_estimator_

DecisionTreeClassifier(max_depth=35, min_samples_split=5, random_state=7)

In [ ]:
import pandas as pd

cv_results = pd.DataFrame(grid_search.cv_results_)[['params', 'mean_train_score', 'mean_test_score']]
cv_results["diff, %"] = 100 * (cv_results["mean_train_score"] - cv_results["mean_test_score"]) / cv_results["mean_train_score"]

# Extracting and adding decision tree parameters
cv_results['max_depth'] = [params['max_depth'] for params in cv_results['params']]
cv_results['min_samples_split'] = [params['min_samples_split'] for params in cv_results['params']]
cv_results['random_state'] = [params['random_state'] if 'random_state' in params else None for params in cv_results['params']]

# Displaying settings
pd.set_option('display.max_colwidth', 100)

# Sorting
cv_results = cv_results.sort_values('mean_test_score', ascending=False)

# Displaying the updated dataframe
cv_results


,params,mean_train_score,mean_test_score,"diff, %",max_depth,min_samples_split,random_state
42,"{'max_depth': 35, 'min_samples_split': 5}",0.840750,0.343114,59.189557,35,5,None
49,"{'max_depth': 40, 'min_samples_split': 5}",0.840750,0.343114,59.189557,40,5,None
35,"{'max_depth': 30, 'min_samples_split': 5}",0.841133,0.342926,59.230419,30,5,None
28,"{'max_depth': 25, 'min_samples_split': 5}",0.831344,0.340191,59.079367,25,5,None
26,"{'max_depth': 20, 'min_samples_split': 30}",0.527530,0.339243,35.692218,20,30,None
21,"{'max_depth': 20, 'min_samples_split': 5}",0.803454,0.338980,57.809647,20,5,None
23,"{'max_depth': 20, 'min_samples_split': 15}",0.619744,0.338825,45.328334,20,15,None
14,"{'max_depth': 15, 'min_samples_split': 5}",0.706340,0.338690,52.050045,15,5,None
31,"{'max_depth': 25, 'min_samples_split': 20}",0.581563,0.338556,41.785254,25,20,None
24,"{'max_depth': 20, 'min_samples_split': 20}",0.573380,0.338513,40.961824,20,20,None


As can be seen from the diff,% column, which displays a sizable percentage difference between training and test scores in numerous configurations, the test scores are significantly lower than the training scores. This discrepancy indicates that the models may be overfitting to the training set; that is, while they function well on the training set, they do not generalize as well to new data.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

# Applying the Decision Tree Classifier
dt = DecisionTreeClassifier(random_state=7)

# Defining a grid of parameters to search over to address overfitting
param_grid = {
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10],
}

# Applying GridSearchCV with the classifier, parameter grid, and desired scoring metric
grid_search = GridSearchCV(dt, param_grid, cv=5, scoring='accuracy', return_train_score=True)

# Fitting GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Converting the grid search results to a DataFrame
cv_results = pd.DataFrame(grid_search.cv_results_)[['params', 'mean_train_score', 'mean_test_score']]
cv_results["diff, %"] = 100 * (cv_results["mean_train_score"] - cv_results["mean_test_score"]) / cv_results["mean_train_score"]

# Adding decision tree parameters for clarity
cv_results['max_depth'] = [params['max_depth'] for params in cv_results['params']]
cv_results['min_samples_split'] = [params['min_samples_split'] for params in cv_results['params']]
cv_results['min_samples_leaf'] = [params['min_samples_leaf'] for params in cv_results['params']]

# Sorting the results by test score to identify the best parameter combinations
cv_results_sorted = cv_results.sort_values('mean_test_score', ascending=False)

# Display the sorted DataFrame with updated settings
pd.set_option('display.max_colwidth', 100)
print(cv_results_sorted)


                                                                params  \
0      {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}   
1      {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5}   
4      {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2}   
5      {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5}   
8      {'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 2}   
..                                                                 ...   
53    {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5}   
52    {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2}   
65  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5}   
69  {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5}   
68  {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2}   

    mean_train_score  mean_test_score    diff, %  max_depth  \
0           0.775941         0.766776   1.181104

In [ ]:
# Defining a More Focused Hyperparameter Grid
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Refining the hyperparameter grid based on previous results
param_grid_refined = {
    'max_depth': [4, 5, 6],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3],
}

# Applying the Decision Tree Classifier
dt_refined = DecisionTreeClassifier(random_state=7)


In [ ]:
# Setting up GridSearchCV with the refined parameter grid
grid_search_refined = GridSearchCV(dt_refined, param_grid_refined, cv=5, scoring='accuracy', return_train_score=True)

grid_search_refined.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=7),
             param_grid={'max_depth': [4, 5, 6], 'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 4, 6]},
             return_train_score=True, scoring='accuracy')

In [ ]:
# Analyzing the Refined Grid Search Results
# Converting the grid search results to a DataFrame
cv_results_refined = pd.DataFrame(grid_search_refined.cv_results_)[['params', 'mean_train_score', 'mean_test_score', 'rank_test_score']]
cv_results_refined["diff, %"] = 100 * (cv_results_refined["mean_train_score"] - cv_results_refined["mean_test_score"]) / cv_results_refined["mean_train_score"]

# Sorting the results by test score to identify the best parameter combinations
cv_results_refined_sorted = cv_results_refined.sort_values('mean_test_score', ascending=False)

# Displaying the sorted DataFrame
print(cv_results_refined_sorted)


                                                             params  \
2   {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 6}   
3   {'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 2}   
4   {'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 4}   
5   {'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 6}   
6   {'max_depth': 4, 'min_samples_leaf': 3, 'min_samples_split': 2}   
7   {'max_depth': 4, 'min_samples_leaf': 3, 'min_samples_split': 4}   
8   {'max_depth': 4, 'min_samples_leaf': 3, 'min_samples_split': 6}   
0   {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}   
1   {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 4}   
16  {'max_depth': 5, 'min_samples_leaf': 3, 'min_samples_split': 4}   
11  {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 6}   
17  {'max_depth': 5, 'min_samples_leaf': 3, 'min_samples_split': 6}   
14  {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 6}   
15  {'

In [ ]:
# Extracting the best estimator
best_dt_model = grid_search_refined.best_estimator_

y_pred_test = best_dt_model.predict(X_test)

from sklearn.metrics import accuracy_score
print("Test Accuracy:", accuracy_score(y_test, y_pred_test))


Test Accuracy: 0.7709443099273607


Model Generalization: With the improved hyperparameters, the model is more capable of generalizing from the training data to unknown data, as evidenced by its accuracy of 77.1% on the test set. This is a good result.

Comparison with Baseline: This accuracy exceed my baseline models' performance.

## Random Forest Model

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, f1_score
import time

start_time = time.time()

# Applying the classifier
rf = RandomForestClassifier(random_state=7, max_depth=40, min_samples_split=5)

hp_grid = {
    'n_estimators': [100, 200, 500],
    'max_features': ["sqrt", 0.5],
    'max_samples': [None, 0.5],
}

# Useing f1_macro score for multi-class classification
grid_search = GridSearchCV(rf, hp_grid, cv=5,
                           scoring='f1_macro',
                           return_train_score=True, verbose=2)

grid_search.fit(X_train, y_train)

# Printing best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

# Calculating and printing execution time
execution_time = time.time() - start_time
print("Execution time in seconds:", execution_time)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END max_features=sqrt, max_samples=None, n_estimators=100; total time=   0.9s
[CV] END max_features=sqrt, max_samples=None, n_estimators=100; total time=   0.8s
[CV] END max_features=sqrt, max_samples=None, n_estimators=100; total time=   0.8s
[CV] END max_features=sqrt, max_samples=None, n_estimators=100; total time=   0.8s
[CV] END max_features=sqrt, max_samples=None, n_estimators=100; total time=   0.8s
[CV] END max_features=sqrt, max_samples=None, n_estimators=200; total time=   1.6s
[CV] END max_features=sqrt, max_samples=None, n_estimators=200; total time=   2.0s
[CV] END max_features=sqrt, max_samples=None, n_estimators=200; total time=   2.1s
[CV] END max_features=sqrt, max_samples=None, n_estimators=200; total time=   1.7s
[CV] END max_features=sqrt, max_samples=None, n_estimators=200; total time=   1.7s
[CV] END max_features=sqrt, max_samples=None, n_estimators=500; total time=   4.2s
[CV] END max_features=sqrt

In [ ]:
grid_search.best_estimator_


RandomForestClassifier(max_depth=40, max_features=0.5, min_samples_split=5,
                       random_state=7)

In [ ]:
grid_search.best_score_

0.31248534701618846

In [ ]:
import pandas as pd

# Converting the GridSearchCV results into a DataFrame
cv_results = pd.DataFrame(grid_search.cv_results_)[['params', 'mean_train_score', 'mean_test_score']]

# Calculating the percentage difference between train and test scores to assess overfitting
cv_results["diff, %"] = 100 * (cv_results["mean_train_score"] - cv_results["mean_test_score"]) / cv_results["mean_train_score"]

# Seting pandas display options to ensure all columns and their content are displayed properly
pd.set_option('display.max_colwidth', 100)

# Sorting the results by the mean test score in descending order to see the best performing parameters first
sorted_cv_results = cv_results.sort_values('mean_test_score', ascending=False)

# Displaying the sorted results
print(sorted_cv_results)


                                                                params  \
6      {'max_features': 0.5, 'max_samples': None, 'n_estimators': 100}   
7      {'max_features': 0.5, 'max_samples': None, 'n_estimators': 200}   
8      {'max_features': 0.5, 'max_samples': None, 'n_estimators': 500}   
0   {'max_features': 'sqrt', 'max_samples': None, 'n_estimators': 100}   
1   {'max_features': 'sqrt', 'max_samples': None, 'n_estimators': 200}   
9       {'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 100}   
2   {'max_features': 'sqrt', 'max_samples': None, 'n_estimators': 500}   
10      {'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 200}   
3    {'max_features': 'sqrt', 'max_samples': 0.5, 'n_estimators': 100}   
4    {'max_features': 'sqrt', 'max_samples': 0.5, 'n_estimators': 200}   
11      {'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 500}   
5    {'max_features': 'sqrt', 'max_samples': 0.5, 'n_estimators': 500}   

    mean_train_score  mean_test_score

Significant Overfitting: The model appears to be overfitted based on the significant differences in the diff, % column (32% - 61%). It doesn't generalize well.
So in this step it is necessory to fix the problem. The first step is extracting feature importances from the best estimator

In [ ]:
# Extracting feature importances from the best estimator
feature_importances = grid_search.best_estimator_.feature_importances_

# Zip together the feature importances with their corresponding feature names and sort them
sorted_importances = sorted(zip(feature_importances, X_train.columns), reverse=True)

# Printing the features and their importances
for importance, feature_name in sorted_importances:
    print(f"{feature_name}: {importance:.3f}")


age_of_driver: 0.253
engine_capacity_cc: 0.212
age_of_vehicle: 0.179
driver_imd_decile: 0.134
vehicle_reference: 0.043
vehicle_type: 0.040
journey_purpose_of_driver: 0.039
driver_home_area_type: 0.037
propulsion_code: 0.029
sex_of_driver: 0.023
vehicle_left_hand_drive: 0.006
towing_and_articulation: 0.005


The driver's age is the most significant factor. It implies that a driver's age has a significant impact on predicting the seriousness of collisions.The vehicle's engine size has a significant impact on the severity of accidents, making it the second most important feature. Larger engines may be associated with faster speeds or more serious collisions.
Age of Vehicle, this is the third most significant factor, indicating that the safety features and vulnerabilities of older and newer cars may differ and have an impact on the outcome of accidents.
Driver IMD Decile: Indicates the socioeconomic standing of the driver's community. It implies that socioeconomic variables may influence the severity of accidents, perhaps as a result of variations in the caliber of vehicles, the state of the roads, or driving habits.
Even though they have lower importance scores, features like vehicle reference, vehicle type, driver's journey purpose, etc., still help the model make predictions. They represent different facets of the car, the reason for the trip, and the traits of the driver.

In [ ]:
grid_search.best_estimator_

RandomForestClassifier(max_depth=40, max_features=0.5, min_samples_split=5,
                       random_state=7)

In [ ]:
grid_search.best_score_

0.31248534701618846

In [ ]:
import pandas as pd

# Listing of selected features based on importance
selected_features = [
    "age_of_driver", "engine_capacity_cc", "age_of_vehicle",
    "driver_imd_decile", "vehicle_reference", "vehicle_type",
    "journey_purpose_of_driver", "driver_home_area_type"
]

# Defining X_selected using the training data and selected features
X_train_selected = trainset_final[selected_features]

y_train = trainset_final['accident_severity']

X_test_selected = testset_final[selected_features]
y_test = testset_final['accident_severity']


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Applying the model
rf_model = RandomForestClassifier(random_state=42)

# Training the model
rf_model.fit(X_train_selected, y_train)

# Making predictions
predictions = rf_model.predict(X_train_selected)

# Calculating and print the accuracy
print("Accuracy on Training Set:", accuracy_score(y_train, predictions))


Accuracy on Training Set: 0.9982949701619779


In [ ]:
# Making predictions on the test set
test_predictions = rf_model.predict(X_test_selected)

# Calculating and printing the accuracy on the test set
print("Accuracy on Test Set:", accuracy_score(y_test, test_predictions))


Accuracy on Test Set: 0.7472154963680387


With an accuracy of roughly 74.72% on the test set and 99.83% on the training set, there is a noticeable performance difference between the two sets of data. This difference shows that instead of generalizing from the underlying patterns to make precise predictions on unseen data, the model might be overfitting to the training data, which means it has learned the training data—including its noise and outliers—too well.

In [ ]:
# adjusting model complexity
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score

# Adjusting the complexity of the model
rf_model_adjusted = RandomForestClassifier(random_state=42, max_depth=10, min_samples_split=5, min_samples_leaf=4, max_features='sqrt', n_estimators=100)

# Evaluating using cross-validation
cv_scores_adjusted = cross_val_score(rf_model_adjusted, X_train_selected, y_train, cv=5, scoring='accuracy')
print(f"Adjusted CV Accuracy: {cv_scores_adjusted.mean():.2f} +/- {cv_scores_adjusted.std():.2f}")


Adjusted CV Accuracy: 0.77 +/- 0.00


In [ ]:
# Defining a grid of hyperparameters to search
param_grid = {
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [50, 100, 200]
}

# Applying the grid search
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy', verbose=1)
grid_search.fit(X_train_selected, y_train)

# Print the best parameters and corresponding score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)


Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
Best score: 0.7717696528215892


In [ ]:
from sklearn.ensemble import RandomForestClassifier

feature_importances = rf_model_regularized.feature_importances_
most_important_features = np.argsort(feature_importances)[::-1][:10]

print("Top 10 most important features:")
for i in most_important_features:
    print(X_train_selected.columns[i], feature_importances[i])


Top 10 most important features:
engine_capacity_cc 0.21334762872065335
journey_purpose_of_driver 0.16957757193578019
age_of_driver 0.16066144465465593
age_of_vehicle 0.15859291670233341
vehicle_type 0.12954970306733804
driver_imd_decile 0.07381654082707216
vehicle_reference 0.06630427811098152
driver_home_area_type 0.028149915981185412


In [ ]:
important_features = [
    'engine_capacity_cc',
    'journey_purpose_of_driver',
    'age_of_driver',
    'age_of_vehicle',
    'vehicle_type',
]

# Creating a new dataset for training and testing using 'important_features'
X_train_important = trainset_final[important_features]
X_test_important = testset_final[important_features]


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Creating and training the model
rf_important = RandomForestClassifier(random_state=42, max_depth=5, min_samples_split=10, min_samples_leaf=8, n_estimators=100)
rf_important.fit(X_train_important, y_train)

# Evaluating using cross-validation
cv_scores_rf_important = cross_val_score(rf_important, X_train_important, y_train, cv=5, scoring='accuracy')
print(f"Random Forest (Important Features) CV Accuracy: {cv_scores_rf_important.mean():.2f} +/- {cv_scores_rf_important.std():.2f}")


Random Forest (Important Features) CV Accuracy: 0.77 +/- 0.00


The accuracy increased by 77. In this step I keep the result and switch to another machine learning model.

In [ ]:
# creating a folder where all trained models will be kept
if not os.path.exists("models"):
    os.makedirs("models")

dump(grid_search.best_estimator_, 'models/rf-clf.joblib')

['models/rf-clf.joblib']

## Support Vector Machines

# Linear SVMs

In [ ]:
import time
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
import pandas as pd
from datetime import timedelta

# Starting the timer
start = time.time()

lsvm = LinearSVC(random_state=7, max_iter=5000)

hp_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],
}

grid_search = GridSearchCV(lsvm, hp_grid, cv=5, scoring='f1_macro',
                           return_train_score=True, verbose=1)
grid_search.fit(X_train, y_train)

# Calculating execution time
execution_time = time.time() - start
print("Execution time HH:MM:SS:", timedelta(seconds=execution_time))


Fitting 5 folds for each of 5 candidates, totalling 25 fits


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

Execution time HH:MM:SS: 0:03:10.344573


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
grid_search.best_estimator_

LinearSVC(C=0.001, max_iter=5000, random_state=7)

In [ ]:
grid_search.best_score_

0.32075782186886087

In [ ]:
# Extracting results into a DataFrame
cv_results = pd.DataFrame(grid_search.cv_results_)

# Selecting desired columns
cv_results = cv_results[['param_C', 'mean_train_score', 'mean_test_score']]

# Calculating the percentage difference between the training and test scores
cv_results["diff, %"] = 100 * (cv_results["mean_train_score"] - cv_results["mean_test_score"]) / cv_results["mean_train_score"]

# Setting display options for better readability
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.precision', 3)

# Sorting the results by test score in descending order
sorted_cv_results = cv_results.sort_values('mean_test_score', ascending=False)

# Displaying the sorted results
display(sorted_cv_results)


,param_C,mean_train_score,mean_test_score,"diff, %"
0,0.001,0.316,0.321,-1.536
1,0.01,0.297,0.297,-0.137
3,1,0.285,0.285,-0.101
2,0.1,0.258,0.257,0.328
4,10,0.253,0.251,0.642


With certain configurations indicating the model performs marginally better on the test set, the differences between test and training scores are generally negligible for a range of C values. This situation typically indicates underfitting, a condition in which the model is too simplistic to adequately represent the underlying patterns in the training and test datasets. so using the radial basis function in the following step is preferable.

In [ ]:
import time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd
from datetime import timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Starting the timer
start = time.time()

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Defining the model using SVC with RBF kernel
svc = SVC(random_state=7, kernel='rbf', max_iter=5000)

# Specify the hyperparameters and their values for the grid search
hp_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1]
}

# Setup GridSearchCV
grid_search = GridSearchCV(svc, hp_grid, cv=5, scoring='f1_macro',
                           return_train_score=True, verbose=1)

# Performing grid search on the scaled data
grid_search.fit(X_train_scaled, y_train)

# Calculating execution time
execution_time = time.time() - start
print("Execution time HH:MM:SS:", timedelta(seconds=execution_time))

# Print best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)


Fitting 5 folds for each of 25 candidates, totalling 125 fits


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider

Execution time HH:MM:SS: 0:07:51.427428
Best parameters: {'C': 10, 'gamma': 1}
Best score: 0.3511104830964304


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [ ]:
grid_search.best_estimator_

SVC(C=10, gamma=1, max_iter=5000, random_state=7)

In [ ]:
grid_search.best_score_

0.3511104830964304

As the result is better I try to check it on test data

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

X_test_scaled = scaler.transform(X_test)
y_pred = grid_search.best_estimator_.predict(X_test_scaled)

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Test F1 Score (Macro):", f1_score(y_test, y_pred, average='macro'))


Test Accuracy: 0.6944309927360774
Test F1 Score (Macro): 0.3564312651506995


Test Accuracy: The model can correctly predict almost 70% of the time based on its accuracy of roughly 69.44% on the test set.
F1 Score (Macro): For a multi-class classification problem, an F1 score of 0.356 is comparatively low. Using a macro average that treats all classes equally, this metric computes the harmonic mean of precision and recall. The model may be doing well overall, but its lower F1 score indicates that it may not be able to accurately predict every class.

In [ ]:
# Using Polinomial SVM

# Starting the timer
start_time = time.time()

# Pipeline for scaling data
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='poly', random_state=7))
])

hp_grid = {
    'svm__C': [0.01, 0.1, 1],
    'svm__degree': [2, 3, 4],
    'svm__coef0': [0, 1]
}

# Seting up GridSearchCV
grid_search_poly = GridSearchCV(pipeline, hp_grid, cv=5, scoring='f1_macro',
                                return_train_score=True, verbose=2)

grid_search_poly.fit(X_train, y_train)

# Calculating execution time
execution_time = time.time() - start_time
print("Execution time HH:MM:SS:", timedelta(seconds=execution_time))

# Print the best parameters and the best score achieved
print("Best parameters:", grid_search_poly.best_params_)
print("Best score:", grid_search_poly.best_score_)


Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END ...........svm__C=0.01, svm__coef0=0, svm__degree=2; total time=   2.2s
[CV] END ...........svm__C=0.01, svm__coef0=0, svm__degree=2; total time=   1.1s
[CV] END ...........svm__C=0.01, svm__coef0=0, svm__degree=2; total time=   1.1s
[CV] END ...........svm__C=0.01, svm__coef0=0, svm__degree=2; total time=   1.1s
[CV] END ...........svm__C=0.01, svm__coef0=0, svm__degree=2; total time=   2.3s
[CV] END ...........svm__C=0.01, svm__coef0=0, svm__degree=3; total time=   2.3s
[CV] END ...........svm__C=0.01, svm__coef0=0, svm__degree=3; total time=   2.7s
[CV] END ...........svm__C=0.01, svm__coef0=0, svm__degree=3; total time=   2.7s
[CV] END ...........svm__C=0.01, svm__coef0=0, svm__degree=3; total time=   3.5s
[CV] END ...........svm__C=0.01, svm__coef0=0, svm__degree=3; total time=   2.6s
[CV] END ...........svm__C=0.01, svm__coef0=0, svm__degree=4; total time=   2.4s
[CV] END ...........svm__C=0.01, svm__coef0=0, s

Model Performance: The Polynomial SVM did not perform better than my previous models, specifically the RBF kernel SVM, which obtained a higher f1_macro score of roughly 0.351, as indicated by the f1_macro score of 0.303. This suggests that the polynomial kernel might not be the best fit for capturing the patterns in the data, or it might require more experimenting, even though it can model more complex relationships than a linear kernel.
Best Parameters: Among the configurations tested, a higher degree polynomial degree=4, with C=1 and coef0=1 appears to have been the most effective. This configuration suggests a somewhat intricate model, but it was still unable to match the RBF kernel SVM's performance.

In [ ]:
grid_search.best_estimator_

SVC(C=10, gamma=1, max_iter=5000, random_state=7)

In [ ]:
cv_results = pd.DataFrame(grid_search.cv_results_)[['params', 'mean_train_score', 'mean_test_score']]
cv_results["diff, %"] = 100*(cv_results["mean_train_score"]-cv_results["mean_test_score"]
                                                     )/cv_results["mean_train_score"]

pd.set_option('display.max_colwidth', 100)
cv_results.sort_values('mean_test_score', ascending=False)

,params,mean_train_score,mean_test_score,"diff, %"
24,"{'C': 10, 'gamma': 1}",0.909,0.351,6.138e+01
23,"{'C': 10, 'gamma': 0.1}",0.435,0.300,3.117e+01
19,"{'C': 1, 'gamma': 1}",0.508,0.298,4.149e+01
20,"{'C': 10, 'gamma': 'scale'}",0.400,0.296,2.583e+01
21,"{'C': 10, 'gamma': 'auto'}",0.400,0.295,2.626e+01
1,"{'C': 0.001, 'gamma': 'auto'}",0.290,0.290,-1.772e-06
18,"{'C': 1, 'gamma': 0.1}",0.292,0.290,5.646e-01
17,"{'C': 1, 'gamma': 0.01}",0.290,0.290,-1.772e-06
16,"{'C': 1, 'gamma': 'auto'}",0.291,0.290,3.001e-01
15,"{'C': 1, 'gamma': 'scale'}",0.291,0.290,2.669e-01


#**Evaluate The Best Model**

In [ ]:
from joblib import load

best_rf = load("models/rf-clf.joblib")

## **Random Forest**

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

best_rf = grid_search.best_estimator_

# Making predictions on the test set
yhat = best_rf.predict(X_test)

# Calculating evaluation metrics
p, r, f, s = precision_recall_fscore_support(y_test, yhat, average="macro")
print("Random Forest (with grid search optimization):")
print(f"Precision: {p}")
print(f"Recall: {r}")
print(f"F score: {f}")


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


Random Forest (with grid search optimization):
Precision: 0.2571428571428572
Recall: 0.3333333333333333
F score: 0.2903225806451613


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#**Possible Future Improvement**

The precision, recall, and F-score values suggest that there's significant room for improving my Random Forest model. Here's a breakdown of what these metrics indicate and potential areas to explore:

Precision 0.257: Out of all the instances my model predicted as positive, only about 25.7% were actually positive.
Recall 0.333: my model is only correctly identifying about 33.3% of the truly positive cases.
F-score 0.290: The low value confirms the need for improvement.

**Possible Reasons & Future Improvements:**

Class Imbalance:  It appears that my dataset has imbalanced classes, it's a major factor affecting model performance.  It should be investigated and addressed by using these techniques:
Resampling,Class Weights( Adjust weights in Random Forest classifier),Feature Engineering (using new Features, Feature Interactions and Feature Transformations).

Data Collection: Collecting more data.


#**Possible scenarios to deploy the models in realworld business scenarios**





**Scenario 1: Insurance Providers**

Risk assessment: By using the models, insurers could estimate the chance of serious accidents involving specific drivers. This facilitates the computation of more precise risk profiles and the establishment of customized insurance rates.
Preventive measures: In order to lower overall risk, insurers may identify high-risk patterns—features linked to serious accidents—and then launch focused awareness campaigns or provide individualized safe-driving programs.

**Scenario 2: Driver Behavior Profiling for Auto Fleets**

Fleet Monitoring: Onboard devices have models that can be used by fleets of vehicles, such as delivery trucks, taxis, and rental cars. This evaluates driving behaviors (such as sudden braking, speeding, and distracted driving patterns) that are linked to an increased risk of accidents.

Coach and feedback for drivers: The system can give drivers instant feedback to promote safer driving practices. Supervisors are also able to recognize drivers who could benefit from specialized training.

Insurance Optimization: Compared to using only traditional factors, fleet insurers can evaluate risk profiles more dynamically with the use of this data. More personalized insurance plans and rewards for safe driving may result from this.